# CS:GO Professional Player Cluster Analysis

The most recent Counter-Strike: Global Offensive (CS:GO) major tournament had a first ever \\$1,000,000 first place prize pool for the winning team with an overall prize pool for all participants of \\$3,000,000. With the stakes in professional play being higher than ever before. I figured it would be pertinent to develop a tool for coaches and team managers to utilize when trying to put together their next million dollar winning team lineup.

The dataset used can be found [here](https://www.kaggle.com/mateusdmachado/csgo-professional-matches). The range of the dataset covers professional matches from 11/2015 to 03/2020.

## What is CS:GO?

CS:GO or Counter-Strike: Global Offensive is the latest entry to the Counter-Strike franchise from Valve. Launched in 2012 it is a 5 versus 5 tactical FPS (First Person Shooter). The two teams face each other in a matchup. The gameplay is as follows, the Terrorist (T) side is tasked with planting a bomb and having it detonate while the Counter-Terrorist side (CT) is to defuse it or prevent it from being planted. Both teams can win a round if they eliminate all players on the opposing team.

A standard game is 30 rounds with the first team to win 16 rounds taking the win for the map with overtime rounds being a possibility. In the event that there is a draw and the match goes into overtime the first team to win two consecutive rounds takes the match.

Counter-strike has an economic system that governs the acquisitions of armor, weapons and grenades by the players. Winning a round award the players with \\$3250 while losing a round after a winning streak gives them \\$1400. Losing many times in a short period increases the losing bonus by \\$500 for every additional loss, as to not penalize the losing team too much. Players can also win money by getting kills and planting or defusing the bomb.

## Project Objective

There are informal roles that each of the 5 players per team take on over the course of a game. I say informal as due to the nature of the game these roles are flexible and a player may be forced to wear multiple hats during the course of a single round. With that said the roles are as follows:

- **Entry Fragger**: The first player to get into a position, whether it be the bombsite, or an objective. These players initiate the push and look to open up a path by securing an entry kill or to provide some kind of an opportunity to make something happen. The success of this role is measured by how often they get the first kill to provide an advantage to their team. Often times these players will also be the first to die.
- **Support**: This player pushes along side the entry fragger and provides cover through the use of utility (grenades). Additionally these players will trade out a kill if the entry fragger falls to an enemy or cover additional lines of sight that the entry fragger cannot hold on their own.
- **AWPer**: This role gets its namesake from the AWP, a high power long range rifle which can be purchased by both teams for \\$4750. This player holds long angles to secure crucial kills as well as to deny space to the enemy team
- **Lurker**: A lurker is a player that typically plays separately from their team, looking to catch an enemy player out of place, in isolation, or transitioning from one bombsite to another. Their primary focus is to catch the other team off guard and prevent them from re-taking a bombsite with ease.
- **IGL (In-Game-Leader)**: As the name suggests this is the player that is calling the shots, they determine how the team uses their economy, the strategies they employ, and just about any other decision that can be made from a macro level over the course of a round.

As one would imagine these different types of players will have varying play styles and statistics over the course of a match. The objective of this project is to see if a players statistics will allow us to determine how similar a given player is to another.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Data Exploration

In [2]:
# Reading in the relevant tables
players_df = pd.read_csv('data/players.csv')
results_df = pd.read_csv('data/results.csv')

These two tables provide us with all the information regarding the players as well as all the matches. The remaining tables in the dataset provide us with map picks and bans, as well as the economy information. This data goes beyond the scope of our project so we won't be using them for anything.

In [3]:
results_df.head()

,date,team_1,team_2,_map,result_1,result_2,map_winner,starting_ct,ct_1,t_2,t_1,ct_2,event_id,match_id,rank_1,rank_2,map_wins_1,map_wins_2,match_winner
0,2020-03-18,Recon 5,TeamOne,Dust2,0,16,2,2,0,1,0,15,5151,2340454,62,63,0,2,2
1,2020-03-18,Recon 5,TeamOne,Inferno,13,16,2,2,8,6,5,10,5151,2340454,62,63,0,2,2
2,2020-03-18,New England Whalers,Station7,Inferno,12,16,2,1,9,6,3,10,5243,2340461,140,118,12,16,2
3,2020-03-18,Rugratz,Bad News Bears,Inferno,7,16,2,2,0,8,7,8,5151,2340453,61,38,0,2,2
4,2020-03-18,Rugratz,Bad News Bears,Vertigo,8,16,2,2,4,5,4,11,5151,2340453,61,38,0,2,2


In [4]:
players_df.head()

,date,player_name,team,opponent,country,player_id,match_id,event_id,event_name,best_of,...,m3_kddiff_ct,m3_adr_ct,m3_kast_ct,m3_rating_ct,m3_kills_t,m3_deaths_t,m3_kddiff_t,m3_adr_t,m3_kast_t,m3_rating_t
0,2020-02-26,Brehze,Evil Geniuses,Liquid,United States,9136,2339385,4901,IEM Katowice 2020,3,...,-1.0,72.5,80.0,0.93,7.0,9.0,-2.0,70.4,63.6,0.89
1,2020-02-26,CeRq,Evil Geniuses,Liquid,Bulgaria,11219,2339385,4901,IEM Katowice 2020,3,...,3.0,79.5,53.3,1.12,4.0,8.0,-4.0,40.7,54.5,0.53
2,2020-02-26,EliGE,Liquid,Evil Geniuses,United States,8738,2339385,4901,IEM Katowice 2020,3,...,1.0,81.5,63.6,1.03,9.0,9.0,0.0,87.9,73.3,1.05
3,2020-02-26,Ethan,Evil Geniuses,Liquid,United States,10671,2339385,4901,IEM Katowice 2020,3,...,0.0,67.2,66.7,0.97,1.0,9.0,-8.0,14.8,45.5,0.31
4,2020-02-26,NAF,Liquid,Evil Geniuses,Canada,8520,2339385,4901,IEM Katowice 2020,3,...,-1.0,72.9,81.8,0.96,8.0,7.0,1.0,56.3,80.0,0.99


### Data Preparation

Much like traditional sports there are a number of statistics tied to each player that allow us to evaluate individual performance. For example:

- Kills
- Assists
- Deaths
- Headshot Percentage
- Flash Assists
- ADR (Average Damage per Round)

These are the key variables which we need to extract from the two tables we've read in.

Our original data set is quite large as seen below.

In [5]:
players_df.shape

(383317, 101)

In [6]:
results_df.shape

(45773, 19)

We have 101 columns in one and 19 in the other, the data is separated out in a way that there are statistics for each map played in a match at the event. We want to summarize the data along with the statistics that measure player performance on a per-match basis.

The code below to extract player statistics by match from our original data tables was referenced from Kaggle user evoluu. The source can be found [here](https://www.kaggle.com/evoluu/csgo-player-statistics-vs-rounds-won/notebook).

The code was modified to include some additional labels.

In [7]:
playerstats_df = pd.DataFrame()

# Join all the individual maps played into one table
for i in [1, 2, 3]:
    player_columns = ['date', 'match_id', 'event_name', 'player_id', 'player_name', 'team', f'map_{i}',
                      f'm{i}_kills', f'm{i}_assists', f'm{i}_deaths', f'm{i}_hs', f'm{i}_flash_assists',
                      f'm{i}_kast', f'm{i}_kddiff', f'm{i}_adr', f'm{i}_fkdiff', f'm{i}_rating']
    temp_df = players_df[player_columns]
    # Rename the columns to exclude map index
    temp_df.columns = ['date', 'match_id', 'event_name', 'player_id', 'player_name', 'team', '_map',
                      'kills', 'assists', 'deaths', 'hs', 'flash_assists',
                      'kast', 'kddiff', 'adr', 'fkdiff', 'rating']
    temp_df = temp_df.dropna()
    playerstats_df = playerstats_df.append(temp_df)

In [8]:
# Add the rounds won/lost data from results_df to our dataframe
teams_df_list = [None, None]
for i in range(2):
    teams_df_list[i] = results_df[['match_id', '_map', f'team_{i+1}', f'result_{i+1}', f'result_{(not i) + 1}']]
    teams_df_list[i].rename(columns={f'team_{i+1}': 'team', f'result_{i+1}': 'rounds_won', f'result_{(not i) + 1}': 'rounds_lost'}, inplace = True) 

teams_df = pd.concat(teams_df_list)
playerstats_df = playerstats_df.merge(teams_df, on=['match_id', '_map', 'team'])

C:\Users\Vincent\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [9]:
# Adding an additional column to identify whether the team won or lost the map
playerstats_df['map_won'] = (playerstats_df['rounds_won'] > playerstats_df['rounds_lost']).astype(int)

In [10]:
playerstats_df.head()

,date,match_id,event_name,player_id,player_name,team,_map,kills,assists,deaths,hs,flash_assists,kast,kddiff,adr,fkdiff,rating,rounds_won,rounds_lost,map_won
0,2020-02-26,2339385,IEM Katowice 2020,9136,Brehze,Evil Geniuses,Overpass,11.0,3.0,18.0,5.0,0.0,65.2,-7.0,60.8,-1.0,0.70,7,16,0
1,2020-02-26,2339385,IEM Katowice 2020,11219,CeRq,Evil Geniuses,Overpass,11.0,2.0,17.0,4.0,2.0,60.9,-6.0,68.9,-1.0,0.75,7,16,0
2,2020-02-26,2339385,IEM Katowice 2020,10671,Ethan,Evil Geniuses,Overpass,11.0,1.0,15.0,6.0,1.0,65.2,-4.0,60.7,-2.0,0.73,7,16,0
3,2020-02-26,2339385,IEM Katowice 2020,8507,stanislaw,Evil Geniuses,Overpass,10.0,1.0,17.0,6.0,0.0,43.5,-7.0,64.5,-4.0,0.65,7,16,0
4,2020-02-26,2339385,IEM Katowice 2020,8523,tarik,Evil Geniuses,Overpass,14.0,7.0,15.0,6.0,3.0,69.6,-1.0,63.4,-1.0,0.95,7,16,0


In [11]:
playerstats_df.shape

(326676, 20)

In [12]:
# Saving to CSV for future usage
# playerstats_df.to_csv("data/player_stats.csv")

#### Data Dictionary

Many of the fields explain themselves however the remainder are somewhat more abstract I will explain them now.

- **HS**: Number of kills that were headshots (single shot kill)
- **flash_assists**: Number of assists that were the result of a flash grenade blinding the enemies
- **kast**: Percentage of rounds in which the player had a kill, assist, survived, or was traded out essentially a measure of player impact
- **kddiff**: Difference between number of kills and number of deaths
- **adr**: Average damage dealt per-round
- **fkdiff**: Number of times the player got the first kill vs the number of times they were first to die, a higher number is better
- **rating**: A rating of the player from HLTV (CS:GO statistic aggregate site). A measure used to assess player performance, calculated with things like damage dealt, opening kills, 1 on X wins, traded deaths and more. The average value for rating is 1.00, good performances will give ratings higher than 1.00 and bad ones below 1.00

Now that we have the data we need, we want to separate out the matches by year into their own dataframes for us to do further work on. The rationale behind this is that we want to see how players are clustered across each year.

In [13]:
# Convert date to datetime
playerstats_df['date'] = pd.to_datetime(playerstats_df['date'])

In [14]:
playerstats_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 326676 entries, 0 to 326675
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   date           326676 non-null  datetime64[ns]
 1   match_id       326676 non-null  int64         
 2   event_name     326676 non-null  object        
 3   player_id      326676 non-null  int64         
 4   player_name    326676 non-null  object        
 5   team           326676 non-null  object        
 6   _map           326676 non-null  object        
 7   kills          326676 non-null  float64       
 8   assists        326676 non-null  float64       
 9   deaths         326676 non-null  float64       
 10  hs             326676 non-null  float64       
 11  flash_assists  326676 non-null  float64       
 12  kast           326676 non-null  float64       
 13  kddiff         326676 non-null  float64       
 14  adr            326676 non-null  float64       
 15  

Nice to know that our dataset does not have any null values as due to the nature of this dataset we would not be able to impute those values reasonably.

In [15]:
# Extracting year into it's own column
playerstats_df['year'] = pd.DatetimeIndex(playerstats_df['date']).year

In [16]:
# Verification
playerstats_df['year'].unique()

array([2020, 2019, 2018, 2017, 2016], dtype=int64)

In [25]:
# Subsetting our data

# Initialize some empty dataframes
stats_2016 = pd.DataFrame()
stats_2017 = pd.DataFrame()
stats_2018 = pd.DataFrame()
stats_2019 = pd.DataFrame()
stats_2020 = pd.DataFrame()

stats_2016 = playerstats_df.loc[playerstats_df['year'] == 2016].reset_index(drop=True)
stats_2017 = playerstats_df.loc[playerstats_df['year'] == 2017].reset_index(drop=True)
stats_2018 = playerstats_df.loc[playerstats_df['year'] == 2018].reset_index(drop=True)
stats_2019 = playerstats_df.loc[playerstats_df['year'] == 2019].reset_index(drop=True)
stats_2020 = playerstats_df.loc[playerstats_df['year'] == 2020].reset_index(drop=True)

In [29]:
# Verification
stats_2016.head()

,date,match_id,event_name,player_id,player_name,team,_map,kills,assists,deaths,...,kddiff,adr,fkdiff,rating,rounds_won,rounds_lost,map_won,year,month,day
0,2016-12-26,2307272,Hellcase Cup 1,11842,Ramz1kBO$$,Tengri,Overpass,35.0,2.0,32.0,...,3.0,72.7,2.0,1.08,22,20,1,2016,12,26
1,2016-12-26,2307272,Hellcase Cup 1,9445,SNk,Tengri,Overpass,33.0,5.0,28.0,...,5.0,84.0,-1.0,1.15,22,20,1,2016,12,26
2,2016-12-26,2307272,Hellcase Cup 1,10052,fitch,Tengri,Overpass,31.0,7.0,32.0,...,-1.0,96.1,-5.0,1.09,22,20,1,2016,12,26
3,2016-12-26,2307272,Hellcase Cup 1,5202,keeN,Tengri,Overpass,21.0,4.0,28.0,...,-7.0,57.6,-1.0,0.81,22,20,1,2016,12,26
4,2016-12-26,2307272,Hellcase Cup 1,9443,pachanga,Tengri,Overpass,26.0,7.0,28.0,...,-2.0,68.3,-7.0,0.98,22,20,1,2016,12,26
